# Banco de dados Solo Criado: Take 1

Vou fazer primeiro no notebook para ir testando, depois passo para o arquivo .py

## Roadmap

* Coletar listagem do site da prefeitura
* Transformar pdf da listagem em xmls por este site
* Transformar xmls em cvs
* Carregar cvs de listagem para o banco de dados
  * Extração;
  * Exploração;
  * Limpeza;
  * Agregação;
  * Visualização;
  * Storytelling.

## Segunda parte: a criação do banco de dados

### 1. Extração

imports

In [9]:
import pandas as pd
import os
import geopandas as gpd
import functools as ft

Testando a função db_manager

In [2]:
from extract_sc import db_manager

db_estq = db_manager("./dados/ListagemSoloCriado/XLSX")


O banco de dados está desatualizado
Adicionando listagem 1
Adicionando listagem 2
Adicionando listagem 3
Adicionando listagem 4
Adicionando listagem 5
Adicionando listagem 6
Adicionando listagem 7
Adicionando listagem 8
Adicionando listagem 9
Adicionando listagem 10
Adicionando listagem 11
Adicionando listagem 12
Adicionando listagem 13
Adicionando listagem 14
Adicionando listagem 15
Adicionando listagem 16


## Terceira parte: Manipulação do Banco de Dados

### Agregação

estoque_maximo

In [4]:
#estq_max_num.to_csv(path_or_buf="./dados/estq_max_num.csv", sep=';') #já rodei o codigo 1 vez

testando a função estoque_maximo

In [3]:
from transform_sc import estoque_maximo, cria_uso_estoque

estq_max_num = estoque_maximo(db_estq)
uso_estq = cria_uso_estoque(estq_max_num)